In [1]:
import tensorflow as tf

import os
print(os.getcwd())

vocab_file = r'.\deep_learn\jd_deep_learn\jd_vocab.txt'


C:\Users\Administrator


In [2]:
class Vocab:
    def __init__(self, filename, num_word_threshold):
        self._word_to_id = {}
        self._unk = -1
        self._num_word_threshold = num_word_threshold
        self._read_dict(filename)
        
    def _read_dict(self, filename):
        with open(filename, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        for line in lines:
            word, frequency = line.strip('\r\n').split('\t')
            frequency = int(frequency)
            if frequency < self._num_word_threshold:
                continue
            idx = len(self._word_to_id)
            if word == '<UNK>':
                self._unk = idx
            self._word_to_id[word] = idx
    def word_to_id(self, word):
        return self._word_to_id.get(word, self._unk)
    
    @property
    def unk(self):
        return self._unk
    
    def size(self):
        return len(self._word_to_id)
    
    def sentence_to_id(self, sentence):
        
        word_ids = [self.word_to_id(cur_word) for cur_word in sentence.split()]
        return word_ids
vocab = Vocab(vocab_file, 10)
vocab_size = vocab.size()
print(vocab_size)
print(vocab.unk)

4761
0


In [3]:
sentence = '首先和评论手机，手机质感和流畅的还是很不错，系统很好就是新系统bug有点多，总体评价来说还是很不错的一个购物外形外观延用了苹果手机的一惯外形和材质，拿在手里很有质感和重量，去实体门店看过了，经常断货，没想到京东有，就果断拍下了！择这款手机的主要原因是它的 屏幕介于苹果11PRO和11PRO MAX 之间，对于我来讲，简直是为我定制的一样，怎样挑剔的我，都无话可说了！屏幕的分辨率也是很棒的，音效我感觉比以前的苹果手机也有进步。拍照效果一如既往的好啊，像素高的超前，还增加了广角拍摄，十分满意！运行速度流畅，反应速度很快，完美！'
import jieba
import numpy as np
sentence_list = jieba.cut(sentence)
words = ' '.join([word for word in sentence_list])
id_words = vocab.sentence_to_id(words)
id_words = id_words[0: 120]
padding_num = 120 - len(id_words)
id_words = id_words + [0 for _ in range(padding_num)]
id_words_list = []
for _ in range(100):
    id_words_list.append(id_words)
# print(np.asarray(id_words_list, dtype=np.int32))
print(np.asarray(id_words_list, dtype=np.int32).shape)
word_np = np.asarray(id_words_list, dtype=np.int32)
simple_word = np.asarray(id_words)
# print(words)
print(simple_word.shape)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.590 seconds.
Prefix dict has been built succesfully.


(100, 120)
(120,)


In [5]:
# lstm_model
lstm_graph_path = r'.\deep_learn\jd_deep_learn\jd_lstm_runout\lstm_model\ckp-10000.meta'
lstm_model_path = r'.\deep_learn\jd_deep_learn\jd_lstm_runout\lstm_model\ckp-10000'
# batch_size = 100
# num_timesteps = 120
# inputs = tf.placeholder(tf.int32, (batch_size, num_timesteps))
# outputs = tf.placeholder(tf.int32, (batch_size, ))
# keep_prob = tf.placeholder(tf.float32, name='keep_prob')
inputs = tf.placeholder(tf.int32, (120))
# outputs = tf.placeholder(tf.int32, (100, ))
with tf.Session() as sess:
    saver = tf.train.import_meta_graph(lstm_graph_path)
    saver.restore(sess, lstm_model_path)
    tensor_inputs = tf.get_default_graph().get_tensor_by_name('Placeholder:0')
    tensor_outpus = tf.get_default_graph().get_tensor_by_name('metrics/ArgMax:0')
    tensor_keep_prob = tf.get_default_graph().get_tensor_by_name('keep_prob:0')
    print(tensor_inputs)
    # acc = tf.argmax(tensor_outpus, 1)
    pred = sess.run(tensor_outpus, feed_dict={inputs: simple_word, tensor_keep_prob:1})
    print(pred[0])
    # print(sess.run())


INFO:tensorflow:Restoring parameters from .\deep_learn\jd_deep_learn\jd_lstm_runout\lstm_model\ckp-10000
Tensor("Placeholder:0", shape=(120, ?), dtype=int32)


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype int32 and shape [100,120]
	 [[node Placeholder_2 (defined at <ipython-input-4-34157cf6dab0>:13) ]]

Caused by op 'Placeholder_2', defined at:
  File "D:\Anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "D:\Anaconda\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "D:\Anaconda\lib\asyncio\base_events.py", line 539, in run_forever
    self._run_once()
  File "D:\Anaconda\lib\asynci

In [47]:
# ckp 转 pb
from tensorflow.python.framework import graph_util
with tf.Session() as sess:
    saver = tf.train.import_meta_graph(lstm_graph_path)
    saver.restore(sess, lstm_model_path)
    
    graph_def = tf.get_default_graph().as_graph_def()
    output_graph_def = graph_util.convert_variables_to_constants(
        sess, graph_def,['metrics/ArgMax']
    )
    with tf.gfile.GFile('.\deep_learn\jd_deep_learn\jd_lstm_runout\lstm_model\jd_comment_lstm_model.pb', 'w') as gf:
        serialized_graph = output_graph_def.SerializeToString()
        gf.write(serialized_graph)

INFO:tensorflow:Restoring parameters from .\deep_learn\jd_deep_learn\jd_lstm_runout\lstm_model\ckp-10000
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 9 variables.
INFO:tensorflow:Converted 9 variables to const ops.


In [51]:
# pb 转serving 的pb
def load_model(model_path):
    if not os.path.exists(model_path):
        raise ValueError("model is not exists")
    model_graph = tf.Graph()
    with model_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(model_path, 'rb') as gf:
            serialized_data = gf.read()
            od_graph_def.ParseFromString(serialized_data)
            tf.import_graph_def(od_graph_def, name='')
    return model_graph

model_path = r'.\deep_learn\jd_deep_learn\jd_lstm_runout\lstm_model\jd_comment_lstm_model.pb'
model_graph = load_model(model_path)
inputs_x = model_graph.get_tensor_by_name('Placeholder:0')
outputs = model_graph.get_tensor_by_name('metrics/ArgMax:0')
keep_prob = model_graph.get_tensor_by_name('keep_prob:0')
with model_graph.as_default():
    with tf.Session(graph=model_graph) as sess:
        for i in model_graph.as_graph_def().node:
            print(i.name)
        pred = sess.run(outputs, feed_dict={inputs_x: word_np, keep_prob:1})
        print(pred[0])


Placeholder
keep_prob
embedding/embedding
embedding/embedding/read
embedding/embedding_lookup/axis
embedding/embedding_lookup
embedding/embedding_lookup/Identity
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/Const
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/Const_1
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/concat/axis
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/concat
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros/Const
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/Const_4
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/Const_5
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/concat_1/axis
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMC